# Semantic search index creation 

### *Guide*

Anton Antonov    
September 2024  

-----

## Introduction

This notebook shows how to create an LLM-computed vector database over the paragraphs of relatively large text.

Here is the Retrieval Augmented Generation (RAG) workflow we consider:

- The document collection is ingested.
- The documents are split into chunks of relevant sizes.
- Large Language Model (LLM) embedding vectors are obtained for all chunks.
- A vector database is created with these embedding vectors and stored locally. Multiple local databases can be created.
- A relevant local database is imported for use.
- An input query is provided to a retrieval system.
- The retrieval system retrieves relevant documents based on the query.
- The top K documents are selected for further processing.
- The model is fine-tuned using the selected documents.
- The fine-tuned model generates an answer based on the query.
- The output answer is presented to the user.

Here is a Mermaid-JS component diagram that shows the components of performing the Retrieval Augmented Generation (RAG) workflow:

```mermaid
flowchart TD
    subgraph LocalVDB[Local Folder]
        A(Vector Database 1)
        B(Vector Database 2)
        C(Vector Database N)
    end
    ID[Ingest document collection]
    SD[Split Documents]
    EV[Get LLM Embedding Vectors]
    CD[Create Vector Database]
    ID --> SD --> EV --> CD

    EV <-.-> LLMs
    
    CD -.- CArray[[CArray<br>representation]]

    CD -.-> |export| LocalVDB

    subgraph Creation
        ID
        SD
        EV
        CD
    end

    LocalVDB -.- JSON[[JSON<br>representation]]

    LocalVDB -.-> |import|D[Ingest Vector Database]
 
    D -.- CArray
    F -.- |nearest neighbors<br>distance function|CArray
    D --> E
    E[/User Query/] --> F[Retrieval]
    F --> G[Document Selection]
    G -->|Top K documents| H(Model Fine-tuning)
    H --> I[[Generation]]
    I <-.-> LLMs
    I -->J[/Output Answer/]
    G -->|Top K passages| K(Model Fine-tuning)
    K --> I

    subgraph RAG[Retrieval Augmented Generation]
        D 
        E
        F
        G
        H
        I
        J
        K
    end
    
    subgraph LLMs
        direction LR
        OpenAI{{OpenAI}}
        Gemini{{Gemini}}
        MistralAI{{MistralAI}}
        LLaMA{{LLaMA}}
    end
```

In this diagram:

- Document collections are ingested, processed, and corresponding vector databases are made.
  - LLM embedding models are used for obtain the vectors.
- There are multiple local vector databases that are stored and maintained locally.
- A vector database from the local collection is selected and ingested.
- An input query provided by the user initiates the RAG workflow.
- The workflow then proceeds with: 
  - retrieval
  - document selection
  - model fine-tuning
  - answer generation
  - presenting the final output


-------

## Setup

In [16]:
use Data::Importers;
use LLM::Functions;
use XDG::BaseDirectory :terms;

use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Reshapers;
use Data::Summarizers;
use JSON::Fast;

In [19]:
my %h = num-type => num32.^name;

to-json(%h)

{
  "num-type": "num32"
}

------

## Ingest text

Ingest the transcript of the (3.5 hours) discussion [CWv2]:

In [2]:
my $url = 'https://podscripts.co/podcasts/modern-wisdom/747-eric-weinstein-why-does-the-modern-world-make-no-sense';
#my $url = 'https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution';
my $txtEN = data-import($url, 'plaintext');

text-stats($txtEN)

(chars => 185217 words => 28485 lines => 4580)

Take the "proper transcript" part:

In [3]:
my $txtEN2 = $txtEN.substr($txtEN.index('Starting point is 00:00:00'));
text-stats($txtEN2)

(chars => 182123 words => 28138 lines => 4522)

Split into paragraphs and make the paragraphs compact:

In [4]:
my @paragraphs = $txtEN2.split(/ 'Starting point is' \h+ [\d ** 2]+ % ':' /):g;
@paragraphs .= map({ $_.subst(/\n+/, "\n"):g});
@paragraphs.elems

284

Show a sample of the paragraphs:

In [5]:
#% html
@paragraphs.pick(4) ==> to-html()

if it's also the case that for such a long time
there hasn't been any progress that's been made,
why are so many people continuing to cling to this?
They're afraid of that man.
That one guy?
Oh yeah.
He's the tyrant that's pulling the strings behind the string.
No, no, no, no, no.
 
 cascade that creates male out of female. Okay, that doesn't always work out. Now you've got an
ambiguous situation, and you've got a culture that basically can't think in ambiguities.
situation and you've got a culture that basically can't think in ambiguities.
That's where a lot of this frustration with the gender binary comes from, is that you know somebody in a category where they're not really one thing or the other at a hardware
level.
I believe that beyond that, there's also a software level.
There are people with male brains and female bodies and conversely, I don't understand this stuff but i believe that that's true if you ever have the
opportunity to interview deidre mccluskey who used to be i think dennis mccluskey
 
 You have no idea what, it's a million years make it to November. 81. Yeah. You have no idea what,
it's a million years between now and November.
I don't know whether Donald Trump
is going to be, you know, facing jail time.
I don't know whether there's going to be
an insurrection by MAGA people
who feel that the Department of Justice
is going after a candidate for political reasons
 
 that's really good yeah yeah i think look i like my ideas being well known there's tons of being
well known that's fun but in the aggregate it's like
somebody tells you you can have you can have an orgasm every three minutes but you can't turn it
off there's some people who have that i know it's a neurological disorder and except it's 30 seconds
right and you can quickly see that you wouldn't sign up for that. Right? And so fame is like that. Do you really want
to never know who sees you when you go out in public?
I've been fascinated by the price that people pay to be someone that most of the world admires.
And Elon was recently on Lex's show and he said, my mind is a storm. I don't think most people

------

## Make vector database

Make an empty vector database object:

In [6]:
my $vdbObj = LLM::RetrievalAugmentedGeneration::VectorDatabase.new(name => 'No833');

VectorDatabase(:id("c3813e9e-b80b-4295-a1b7-4e5ff455b296"), :name("No833"), :elements(0), :sources(0))

Make an LLM access specification:

In [7]:
#my $conf = llm-configuration("ChatGPT", model => 'text-embedding-002');
my $conf = llm-configuration("Gemini");
#my $conf = llm-configuration('LLaMA', model => 'llama-embedding');

$conf.Hash.elems

24

Create the semantic index for the vector database object (an profile it):

In [22]:
my $tstart = now;
$vdbObj.create-semantic-search-index(@paragraphs, method => 'by-max-tokens', max-tokens => 8192, e => $conf):embed;
my $tend = now;
say "Time to make the semantic search index: {$tend - $tstart} seconds.";

Time to make the semantic search index: 200.82027694 seconds.


By default the vector database object is exported in a sub-directory of [`$XDG_DATA_HOME`](https://specifications.freedesktop.org/basedir-spec/latest/index.html):

In [23]:
# The sub-directory
my $dirname = data-home.Str ~ '/raku/LLM/SemanticSearchIndex';

# The exported vector database base file name
my $basename = "SemSe-{$vdbObj.id}.json";

# Corresponding IO:Path object
my $file = IO::Path.new(:$dirname, :$basename);

# Check for existence
$file.f

True

The export path is saved in the vector database object:

In [24]:
$file.Str eq $vdbObj.location

True

Show a sample of the text chunks:

In [25]:
#% html
$vdbObj.items.pairs.pick(4).sort(*.key) ==> to-html()

001.0 What's happening, people? Welcome back to the show. My guest today is Eric Weinstein. He's a mathematician, economist, former managing director at Thiel Capital, and a podcaster. It's a rough time to be a human. The massive increase in information we have access to has made understanding the world harder, not easier. Whether it's higher education, culture, physics, or pretty much anything else, life can be very confusing. Expect to learn what Eric thinks about the most recent furor coming out of Harvard, why the world of physics has made no progress in decades, why Jeffrey Epstein was interested in Stephen Hawking, how much truth there is in the recent flurry of stories about interdimensional extraterrestrials, Eric's predictions for the 2024 035.0 It's going to be a civil war on the faculty. I was learning about an idea, the Abilene Paradox. One of my favorite ideas from last year the abilene paradox is a situation in which a group makes a decision that is contrary to the desires of the group's members because each member assumes the others approve of it that explains how a number of accurate individuals can become idiots when they get together kind of like the emperor's new clothes an acquaintance invites you to his wedding despite not wanting you there because he thinks you want to attend you attend despite not wanting to because you think he wants you there because he thinks you want to attend. You attend despite 085.0 on the world's most important problem, we're going to make you poor. We're going to discredit you. It's almost like there's a force field trying to get us not to unlock this power. And I've been very curious about why that is. And nobody, like, with all the rich people in the world, nobody's funding the stuff at the level that it needs to be funded. This is the most important funding priority on planet Earth. Because otherwise you're all sharing one atmosphere with a bunch of idiots and really powerful toys. So unless we can somehow channel the technology of interdimensional space beings 120.0 So for example, you're not allowed to say that you like cleavage, but you are allowed to say that was an incredibly dramatic neckline right and so why is it that one phrase is penalized it's because there's a russell conjugation that works a russell conjugation that doesn't he sweats she perspires they glow right and so in such circumstances the key question um is how you are allowed to discuss the truth as well as whether you are allowed to discuss the truth. Many times there's a penalty for not being skilled. The skilled person is allowed to say something deftly. The appropriate nuance with appropriate social graces.

Show dimensions and data type of the obtained vectors:

In [26]:
say "dimensions : ", $vdbObj.vectors.&dimensions;
say "data type  : ", deduce-type($vdbObj.vectors);

dimensions : (283 4096)
data type  : Assoc(Vector(Atom((Str)), 283), Tuple([(Any) => 283], 283), 283)


-----

## Summaries

Skim vector databases from the default directory and show summaries:

In [33]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model created>;
vector-database-objects(f=>'hash', :flat)
==> { $_.map({ $_<created> = $_<file>.IO.created.DateTime.Str.subst('T',' ').substr(^19); $_}).sort(*<created>).reverse }()
==> to-html(:@field-names)

id name item-count dimension version llm-service llm-embedding-model created f00209fa-0f35-41bf-8597-744dae9aab5d No833 283 4096 0 llama text-embedding-3-small 2024-09-19 01:31:12 a6b12f80-0ed7-4125-9e2b-9f77c11a16f8 No833 441 4096 0 llama text-embedding-3-small 2024-09-19 01:26:58 32bd53cb-7171-4d26-9a4f-11de377d3af1 words 20 4096 0 llama text-embedding-3-small 2024-09-19 01:14:27 5097c865-7fad-43d9-b7ca-456220d754b7 words 20 768 0 gemini embedding-001 2024-09-18 14:08:44 045f467c-193f-4df6-bec3-790d6c83ca64 No833 441 768 0 gemini embedding-001 2024-09-16 22:21:25 d2effebc-2cef-4b2b-84ca-5dcfa3c1864b No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:30 5cb40fbb-9f69-48ca-9fc1-03ec8059ed99 No747 283 768 0 gemini embedding-001 2024-09-12 13:32:23 44f19858-730e-4b96-86b7-81e701f9df8f No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:18 266b20ca-d917-4ac0-9b0a-7c420625666c No833 441 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:11

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[CWv1] Chris Williamson,
["Eric Weinstein - Why Does The Modern World Make No Sense? (4K)"](https://www.youtube.com/watch?v=p_swB_KS8Hw),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/747-eric-weinstein-why-does-the-modern-world-make-no-sense).)

[CWv2] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)